# Reading a 2D gmsh file
Reading a 2D .msh file with material IDs and boundary IDs.
The unstructured mesh was created using the [gmsh Mesh Generator](https://gmsh.info/).

To run the code, simply type: `jupyter nbconvert --to python --execute <basename>.ipynb`.

To convert it to a python file (named `<basename>.py`), simply type: `jupyter nbconvert --to python <basename>.ipynb`

To run the python file from the terminal, using N processes, simply type: `mpiexec -n <N> python <basename>.py`

In [ ]:
import os
import sys
from mpi4py import MPI

sys.path.append("../..")

from pyopensn.mesh import FromFileMeshGenerator, PETScGraphPartitioner
from pyopensn.context import UseColor, Finalize

UseColor(False)

## Mesh and  Partition
We use the ```FromFileMeshGenerator``` and pass the path to the msh file.

We also partition the 2D mesh into 8 subdomains using `Parmetis` (by running using `mpiexec -n 8 ...`).

Finally, we export the mesh to a VTU file.

In [ ]:
filepath = "../../test/python/modules/linear_boltzmann_solvers/transport_keigen/c5g7/"\
           + "mesh/2D_c5g7_coarse.msh"
meshgen = FromFileMeshGenerator(
    filename=filepath,
    partitioner=PETScGraphPartitioner(type='parmetis'),
)
grid = meshgen.Execute()

# Export
grid.ExportToPVTU("c5g7_mesh_only")

## Visualization
The resulting mesh and material layout is shown below:

![Mesh_Material](images/c5g7_coarse_material.png)

When using the Parmetis partitioner and 8 MPI ranks, we obtain:

![Mesh_Partition](images/c5g7_coarse_partition.png)

## Finalize (for Jupyter Notebook only)

In Python script mode, PyOpenSn automatically handles environment termination. However, this
automatic finalization does not occur when running in a Jupyter notebook, so explicit finalization
of the environment at the end of the notebook is required. Do not call the finalization in Python
script mode, or in console mode.

Note that PyOpenSn's finalization must be called before MPI's finalization.


In [ ]:
from IPython import get_ipython

def finalize_env():
    Finalize()
    MPI.Finalize()

ipython_instance = get_ipython()
if ipython_instance is not None:
    ipython_instance.events.register("post_execute", finalize_env)